In [15]:
from evotorch import Problem
from evotorch.algorithms import GeneticAlgorithm
from evotorch.decorators import vectorized
from evotorch.operators import SimulatedBinaryCrossOver

import torch

In [54]:
@vectorized
def fitness(x: torch.Tensor) -> torch.Tensor:
    print(x.shape)
    print(torch.linalg.norm(x, dim=-1).shape)
    return torch.linalg.norm(x, dim=-1)

In [57]:
fitness(torch.rand(10,3))

torch.Size([10, 3])
torch.Size([10])


tensor([0.6599, 0.7083, 1.1654, 0.6611, 0.4956, 1.0665, 0.8189, 0.6513, 1.0573,
        1.2272])

In [17]:
problem = Problem(
    "min",
    fitness,
    solution_length=5,
    bounds=(
        [-1, -2, -100, -1000, 10.0],
        [1, 2, 100, 1000, 10.5],
    ),
    dtype=torch.float32,
)

problem

[2023-03-04 05:01:18] INFO     < 3590> evotorch.core: Instance of `Problem` (id:140188358984704) -- The `dtype` for the problem's decision variables is set as torch.float32
[2023-03-04 05:01:18] INFO     < 3590> evotorch.core: Instance of `Problem` (id:140188358984704) -- `eval_dtype` (the dtype of the fitnesses and evaluation data) is set as torch.float32
[2023-03-04 05:01:18] INFO     < 3590> evotorch.core: Instance of `Problem` (id:140188358984704) -- The `device` of the problem is set as cpu
[2023-03-04 05:01:18] INFO     < 3590> evotorch.core: Instance of `Problem` (id:140188358984704) -- The number of actors that will be allocated for parallelized evaluation is 0


In [41]:
def my_manual_mutation(x: torch.Tensor) -> torch.Tensor:
    # The default GaussianMutation of EvoTorch does not (yet) support different standard deviation values
    # per variable. However, we can define our own mutation operator which adds noise of different magnitudes
    # to different variables, like shown here:
    [_, solution_length] = x.shape
    dtype = x.dtype
    device = x.device

    # Generate Gaussian noise where each column has its own magnitude
    noise = (
        torch.randn(solution_length, dtype=dtype, device=device)
        * torch.tensor([1, 2, 100, 1000, 0.1], dtype=dtype, device=device)
    )

    result = x + noise

    # Because this is a manual operator, we need to specify how it should respect the strict boundaries.
    # In the case of this example, we just clip the values.
    result = torch.minimum(result, problem.upper_bounds)
    result = torch.maximum(result, problem.lower_bounds)

    return result

In [47]:
searcher = GeneticAlgorithm(
    problem,
    popsize=100,
    operators=[
        SimulatedBinaryCrossOver(problem, tournament_size=4, eta=8),
        my_manual_mutation,
    ],
)
searcher.run(100)

In [43]:
searcher.step()  # Take just one step. Just to see how the population looks like after one generation.

In [48]:
list(searcher.population[:10])

[<Solution values=tensor([ 0.2883,  0.3074, -1.7074, -2.6299, 10.4472]), evals=tensor([10.9157])>,
 <Solution values=tensor([ 0.2883,  0.3085, -2.0354, -2.6440, 10.4471]), evals=tensor([10.9751])>,
 <Solution values=tensor([ 0.2883,  0.3085, -2.0354, -2.6440, 10.4471]), evals=tensor([10.9751])>,
 <Solution values=tensor([ 0.2808,  0.3539,  2.2003, -2.6278, 10.4391]), evals=tensor([10.9966])>,
 <Solution values=tensor([ 1.0000,  0.3212,  5.9355, -0.3956, 10.2478]), evals=tensor([11.8957])>,
 <Solution values=tensor([ 1.0000,  1.4753,  5.9355, -0.8265, 10.4554]), evals=tensor([12.1822])>,
 <Solution values=tensor([-0.8532, -0.4101, -2.3775, -7.4298, 10.4758]), evals=tensor([13.0955])>,
 <Solution values=tensor([ 0.2842,  0.2693, -7.0651, -4.3335, 10.4289]), evals=tensor([13.3270])>,
 <Solution values=tensor([-0.8891, -0.5546,  3.4913, -7.4488, 10.4728]), evals=tensor([13.3586])>,
 <Solution values=tensor([ 0.2883,  0.3085, -6.8905, -4.6530, 10.4527]), evals=tensor([13.3629])>]